<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       OneHotEncodingFit and OneHotEncodingTransform function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The OneHotEncodingFit function outputs a DataFrame of attributes and categorical values to input to OneHotEncodingTransform function, which encodes them as one-hot numeric vectors.
Function encodes specified attributes and categorical values as one-hot numeric vectors,
using OneHotEncodingFit function output.
<br> In this notebook we will see how we can use the OneHotEncodingFit and OneHotEncodingTransform functions available in Vantage.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_OneHotEncodingFitandTransform_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_BankChurn_cloud');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_BankChurn_local');" 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_BankChurn", "customer_churn"))
print("Shape of the data: ", tdf.shape)
tdf

In [ ]:
tdf.tdtypes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A bank wants to analyze whether gender influences customer churn. 
Since "Gender" is a categorical variable (Male/Female), we need to convert it into a numerical form using One-Hot Encoding.
</p>

In [ ]:
help(OneHotEncodingFit)

In [ ]:
# Fit One-Hot Encoding model for the 'Gender' column
fit_result = OneHotEncodingFit(data=tdf,
                               is_input_dense=True,
                               target_column="Gender",
                               categorical_values=["Male", "Female"])

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Transform the Data using the Fitted Encoding
</p>

In [ ]:
help(OneHotEncodingTransform)

In [ ]:
# Applying OneHotEncodingTransform
transformed_data = OneHotEncodingTransform(data=tdf,
                                           object=fit_result.result,
                                           is_input_dense=True)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Gender column is now split into Gender_Male, Gender_Female and Gender_other columns.
</p>

In [ ]:
# Print the transformed DataFrame
transformed_data.result

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Cleanup</b>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_BankChurn');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>

- `RowNumber`: Row index
- `CustomerId`: Unique customer ID
- `Surname`: Customer's surname
- `CreditScore`: Credit score of the customer
- `Geography`: Country (Germany / France / Spain)
- `Gender`: Gender (Male / Female)
- `Age`: Age of the customer
- `Tenure`: Number of years the customer has been associated with the bank
- `Balance`: Account balance
- `NumOfProducts`: Number of bank products used
- `HasCrCard`: Credit card status (0 = No, 1 = Yes)
- `IsActiveMember`: Active membership status (0 = No, 1 = Yes)
- `EstimatedSalary`: Estimated salary of the customer
- `Exited`: Customer churn status (0 = No, 1 = Yes)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>OneHotEncodingFit function reference: <a href = 'https://docs.teradata.com/search/all?query=OneHotEncodingFit&value-filters=prodname~%2522Teradata+Package+for+Python%2522*vrm_release~%252220.00.00.03%2522&content-lang=en-US'>here</a></li>
    <li>OneHotEncodingTransform function reference: <a href = 'https://docs.teradata.com/search/all?query=OneHotEncodingTransform&value-filters=prodname~%2522Teradata+Package+for+Python%2522*vrm_release~%252220.00.00.03%2522&content-lang=en-US'>here</a></li></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>